## 1. Download the Point Cloud Samples from the OpenTrench3D Dataset: 
1. Click link to Kaggle below
2. Log in to Kaggle
3. Download the samples (see the red arrow in below image)
4. unzip the samples folder

This will download a zip file containing two point clouds (.ply) with an compressed/uncompressed size of 45.3MB/182.6MB 

https://www.kaggle.com/datasets/hestogpony/opentrench3d?select=OpenTrench3D_Samples

![download-samples](imgs/download-samples.png "download-samples")

## 2. Install Python Dependencies


```bash
$ pip install -r requirements.txt     # this will install jupyter notebook and Open3D
```


In [ ]:
import os

samples_dir = "path/to/samples/directory"

assert os.path.exists(samples_dir), "samples_dir not found, make sure to point samples_dir to the correct path you just downloaded and unzipped"
assert not samples_dir.endswith(".zip"), "Make sure to unzip the samples directory first"
assert len(os.listdir(samples_dir)) == 2, "Expected 2 files in samples directory, make sure to follow the instructions above"

# Visually Inspect a Point Cloud

In [ ]:
point_cloud_name = "Heating_Area_1_Site_2_part_4.ply" # Heating_Area_1_Site_2_part_4.ply or Water_Area_3_Site_02.ply
point_cloud_fp = os.path.join(samples_dir, point_cloud_name)

assert os.path.exists(point_cloud_fp), "Point cloud file not found. Expected to find it at: {}".format(point_cloud_fp)


In [ ]:
import numpy as np

def load_ply_file(filename):
    """
    Load a PLY file and return the points data as a numpy array.
    """
    with open(filename, 'r') as file:
        # Skip the header
        while True:
            line = file.readline()
            if line.startswith("end_header"):
                break
        
        # Read points data
        points_data = []
        for line in file:
            parts = line.split()
            if len(parts) < 7:  # Ensure there are enough values in the line
                continue
            x, y, z, r, g, b, class_id = map(float, parts[:7])
            points_data.append((float(x), float(y), float(z), int(r), int(g), int(b), int(class_id)))  # Convert to integers

    return np.array(points_data)

In [ ]:
import open3d as o3d

points = load_ply_file(point_cloud_fp)
rgb_points = points[:, :3].copy()
class_points = points[:, :3].copy()

# Subtract/Add half the length from the x-dimension to center the point clouds
# Calculate the length of the point cloud (x-dimension)
length = np.max(points[:, 0]) - np.min(points[:, 0])
rgb_points[:, 0] -= length / 2
class_points[:, 0] += length / 2

# Set the RGB colors
rgb_colors = points[:, 3:6] / 255  # Normalize colors to [0, 1]

# Set the class colors
# Convert the class_id to a colors
class_id_2_class_color_dict = {
    0: np.array([50, 205, 50]),     # Main utility
    1: np.array([255, 223, 0]),     # Other utility
    2: np.array([139, 69, 19]),     # Trench
    3: np.array([85, 85, 85]),      # Inactive utility
    4: np.array([215, 215, 215])    # Misc. 
}
class_colors = np.array([class_id_2_class_color_dict[class_id] / 255 for class_id in points[:, 6]])

# Stack the rgb points and colors points
points = np.vstack((rgb_points, class_points))

# Stack the rgb colors and class colors
colors = np.vstack((rgb_colors, class_colors))

# Add the points and colors to the point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])
